In [ ]:
#8.2 StdOutputCallbackHandler

from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

os.environ['OPENAI_API_KEY'] = ''

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("What is the capital of {country}?")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country='South Africa',callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of South Africa?

> Finished chain.


'\n\nThe capital of South Africa is Pretoria.'

In [ ]:
#8.3 FileHandler
!pip install loguru

from langchain.callbacks import FileCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from loguru import logger

logfile = "output.log"

logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)

llm = OpenAI()
prompt = PromptTemplate.from_template("What is the capital of {country}? ")

# this chain will both print to stdout (because verbose=True) and write to 'output.log'
# if verbose=False, the FileCallbackHandler will still write to 'output.log'
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler], verbose=True)
answer = chain.run(country='Argentina')
logger.info(answer)



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Argentina? 


2023-12-06 05:01:30.568 | INFO     | __main__:<cell line: 19>:19 - 

The capital of Argentina is Buenos Aires.



> Finished chain.


In [ ]:
#8.4 Custom Callbacks

from langchain.callbacks.base import BaseCallbackHandler

class MyHandler(BaseCallbackHandler):
    """Base callback handler that can be used to handle callbacks from langchain."""

    def on_llm_start(self, serialized: 'Dict[str, Any]', prompts: 'List[str]', *, run_id: 'UUID', parent_run_id: 'Optional[UUID]' = None, tags: 'Optional[List[str]]' = None, metadata: 'Optional[Dict[str, Any]]' = None, **kwargs: 'Any'):
        print('We are starting of with this prompt -->',prompts)


    def on_llm_end(self, response: 'LLMResult', *, run_id: 'UUID', parent_run_id: 'Optional[UUID]' = None, **kwargs: 'Any'):
        print('The LLM is done',response)


handler = MyHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("What is the capital of {country}?")

# Request callbacks: Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country='South Africa',callbacks=[handler])


We are starting of with this prompt --> ['What is the capital of South Africa?']
The LLM is done generations=[[Generation(text='\n\nThe capital of South Africa is Pretoria.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 8, 'total_tokens': 19, 'completion_tokens': 11}, 'model_name': 'text-davinci-003'} run=None


'\n\nThe capital of South Africa is Pretoria.'